# ✈️ Inflight Selfie Generator - Training Notebook

This notebook trains an AI model to generate realistic inflight selfies using:
- **IP-Adapter-FaceID** for face-preserving image generation
- **Stable Diffusion XL** as the base model
- **TinyLlama** for intelligent scene planning
- **InsightFace** for face embedding extraction

## Hardware Requirements
- Google Colab with T4 GPU (FREE tier)
- ~15GB VRAM

## Training Time
- IP-Adapter setup: ~10 minutes
- TinyLlama fine-tuning: ~20-30 minutes
- Total: ~40 minutes on T4

## Phase 1: Environment Setup

In [ ]:
# Cell 1: Install Core Dependencies
print("📦 Installing dependencies...")

!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate safetensors
!pip install -q opencv-python-headless pillow
!pip install -q insightface onnxruntime-gpu
!pip install -q huggingface_hub
!pip install -q peft  # For LoRA fine-tuning
!pip install -q bitsandbytes  # For 4-bit quantization
!pip install -q datasets trl  # For training
!pip install -q gradio  # For testing interface

print("✅ Dependencies installed!")

In [ ]:
# Cell 2: Verify GPU
import torch

print("🔍 Checking GPU availability...")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("⚠️ WARNING: No GPU detected! This notebook requires a GPU.")
    print("Please enable GPU: Runtime → Change runtime type → T4 GPU")

In [ ]:
# Cell 3: Download Required Models
from huggingface_hub import hf_hub_download, snapshot_download
import os

print("📥 Downloading models...")

# Create model directories
os.makedirs("models/ip-adapter-faceid", exist_ok=True)
os.makedirs("models/insightface/models/antelopev2", exist_ok=True)

# Download IP-Adapter-FaceID models
print("  Downloading IP-Adapter-FaceID...")
hf_hub_download(
    repo_id="h94/IP-Adapter-FaceID",
    filename="ip-adapter-faceid_sdxl.bin",
    local_dir="./models/ip-adapter-faceid"
)

hf_hub_download(
    repo_id="h94/IP-Adapter-FaceID",
    filename="ip-adapter-faceid-plusv2_sdxl.bin",
    local_dir="./models/ip-adapter-faceid"
)

# Download InsightFace models for face embedding
print("  Downloading InsightFace models...")
insightface_models = [
    "1k3d68.onnx",
    "2d106det.onnx",
    "genderage.onnx",
    "glintr100.onnx",
    "scrfd_10g_bnkps.onnx"
]

for model_file in insightface_models:
    !wget -q -O models/insightface/models/antelopev2/{model_file} \
        https://huggingface.co/datasets/Gourieff/ReActor/resolve/main/models/insightface/models/antelopev2/{model_file}

print("✅ All models downloaded!")

## Phase 2: IP-Adapter-FaceID Pipeline

In [ ]:
# Cell 4: Core Pipeline Implementation
import torch
import cv2
import numpy as np
from PIL import Image
from diffusers import StableDiffusionXLPipeline, DDIMScheduler
from insightface.app import FaceAnalysis
from typing import List, Union

class InflightSelfiePipeline:
    """IP-Adapter-FaceID pipeline for generating inflight selfies."""
    
    def __init__(self, device="cuda"):
        self.device = device
        self.face_analyzer = None
        self.pipe = None
        
        print("🚀 Initializing Inflight Selfie Pipeline...")
        self.setup_face_analyzer()
        self.setup_diffusion_pipeline()
        print("✅ Pipeline ready!")
    
    def setup_face_analyzer(self):
        """Initialize InsightFace for face embedding extraction."""
        print("  Loading face analyzer...")
        self.face_analyzer = FaceAnalysis(
            name='antelopev2',
            root='./models/insightface',
            providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
        )
        self.face_analyzer.prepare(ctx_id=0, det_size=(640, 640))
        print("    ✓ Face analyzer ready")
    
    def setup_diffusion_pipeline(self):
        """Initialize SDXL with IP-Adapter-FaceID."""
        print("  Loading SDXL pipeline...")
        
        # Load base SDXL
        self.pipe = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            torch_dtype=torch.float16,
            variant="fp16",
        ).to(self.device)
        
        # Use efficient scheduler
        self.pipe.scheduler = DDIMScheduler.from_config(self.pipe.scheduler.config)
        
        # Load IP-Adapter-FaceID
        print("    Loading IP-Adapter-FaceID...")
        self.pipe.load_ip_adapter(
            "h94/IP-Adapter-FaceID",
            subfolder=None,
            weight_name="ip-adapter-faceid_sdxl.bin",
        )
        
        # Enable memory optimizations for T4
        self.pipe.enable_model_cpu_offload()
        self.pipe.enable_vae_slicing()
        
        print("    ✓ SDXL + IP-Adapter-FaceID ready")
    
    def extract_face_embedding(self, image: Union[str, np.ndarray, Image.Image]) -> np.ndarray:
        """Extract face embedding from image."""
        # Convert to numpy array
        if isinstance(image, str):
            img = cv2.imread(image)
        elif isinstance(image, Image.Image):
            img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        else:
            img = image
        
        faces = self.face_analyzer.get(img)
        
        if not faces:
            raise ValueError("No face detected in image")
        
        # Return the embedding of the largest face
        face = max(faces, key=lambda x: (x.bbox[2] - x.bbox[0]) * (x.bbox[3] - x.bbox[1]))
        return face.embedding
    
    def extract_face_embeddings_multi(self, images: List[Union[str, np.ndarray, Image.Image]]) -> np.ndarray:
        """Extract and average embeddings from multiple images."""
        embeddings = []
        
        for img in images:
            try:
                emb = self.extract_face_embedding(img)
                embeddings.append(emb)
            except ValueError as e:
                print(f"    ⚠️ Warning: {e}")
                continue
        
        if not embeddings:
            raise ValueError("No faces detected in any images")
        
        # Average the embeddings
        return np.mean(embeddings, axis=0)
    
    def generate_selfie(
        self,
        person1_images: List[Union[str, np.ndarray, Image.Image]],
        person2_images: List[Union[str, np.ndarray, Image.Image]],
        prompt: str,
        negative_prompt: str = None,
        num_inference_steps: int = 30,
        guidance_scale: float = 7.5,
        ip_adapter_scale: float = 0.6,
        seed: int = None,
    ) -> Image.Image:
        """
        Generate inflight selfie with two people.
        
        Args:
            person1_images: List of images for person 1 (paths, arrays, or PIL Images)
            person2_images: List of images for person 2
            prompt: Scene description
            negative_prompt: What to avoid
            num_inference_steps: Diffusion steps (higher = better quality, slower)
            guidance_scale: CFG scale (higher = more prompt adherence)
            ip_adapter_scale: Identity preservation strength (0-1)
            seed: Random seed for reproducibility
        
        Returns:
            Generated PIL Image
        """
        
        # Extract face embeddings
        print("  Extracting face embeddings...")
        emb1 = self.extract_face_embeddings_multi(person1_images)
        emb2 = self.extract_face_embeddings_multi(person2_images)
        
        # Combine embeddings (weighted average)
        # This is a simplified approach - for production, consider:
        # 1. Generating two separate images and compositing
        # 2. Using IP-Adapter-FaceID-Plus for multi-face support
        combined_emb = (emb1 + emb2) / 2
        face_emb_tensor = torch.tensor(combined_emb, dtype=torch.float16).unsqueeze(0).to(self.device)
        
        # Default negative prompt
        if negative_prompt is None:
            negative_prompt = (
                "ugly, blurry, low quality, distorted face, bad anatomy, "
                "deformed, disfigured, watermark, text, oversaturated, "
                "extra limbs, missing limbs, floating limbs, mutation, "
                "duplicate faces, bad eyes, asymmetric eyes"
            )
        
        # Set IP-Adapter scale
        self.pipe.set_ip_adapter_scale(ip_adapter_scale)
        
        # Set seed if provided
        generator = None
        if seed is not None:
            generator = torch.Generator(device=self.device).manual_seed(seed)
        
        # Generate
        print("  Generating image...")
        result = self.pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            ip_adapter_image_embeds=[face_emb_tensor],
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=generator,
            height=1024,
            width=1024,
        )
        
        print("  ✅ Generation complete!")
        return result.images[0]


# Initialize the pipeline
print("\n" + "="*60)
pipeline = InflightSelfiePipeline()
print("="*60 + "\n")

## Phase 3: TinyLlama Scene Planner

In [ ]:
# Cell 5: Scene Planning Dataset
import json

# Training data for scene planning
scene_planning_data = [
    {
        "instruction": "Two friends taking a sunset window selfie flying to Dubai",
        "output": json.dumps({
            "prompt": "two friends taking selfie in airplane window seat, golden sunset visible through window, Dubai skyline approaching in distance, warm golden lighting, happy excited expressions, economy cabin, airplane interior, high quality photo, realistic, detailed faces",
            "negative_prompt": "ugly, blurry, distorted, bad anatomy, deformed faces",
            "ip_adapter_scale": 0.65,
            "guidance_scale": 7.5,
            "scene_type": "sunset_window",
        }, indent=2)
    },
    {
        "instruction": "Business class celebration with champagne",
        "output": json.dumps({
            "prompt": "two people taking selfie in business class airplane cabin, champagne glasses in hand, celebrating, luxury wide seats, premium cabin interior, soft ambient lighting, joyful expressions, high quality photo, detailed, realistic",
            "negative_prompt": "ugly, blurry, economy class, cheap, low quality",
            "ip_adapter_scale": 0.6,
            "guidance_scale": 7.0,
            "scene_type": "business_celebration",
        }, indent=2)
    },
    {
        "instruction": "Night flight with city lights below",
        "output": json.dumps({
            "prompt": "two people taking selfie in airplane at night, city lights visible through window below, dim cabin lighting with city glow on faces, amazed expressions, window seat, high quality photo, realistic, atmospheric",
            "negative_prompt": "daylight, bright, ugly, blurry, bad lighting",
            "ip_adapter_scale": 0.7,
            "guidance_scale": 7.5,
            "scene_type": "night_city",
        }, indent=2)
    },
    {
        "instruction": "Morning flight over clouds",
        "output": json.dumps({
            "prompt": "two people taking selfie in airplane window seat, fluffy white clouds visible outside, bright morning sunlight, fresh morning mood, happy smiling faces, economy cabin, high quality photo, realistic",
            "negative_prompt": "dark, night, ugly, blurry, bad quality",
            "ip_adapter_scale": 0.65,
            "guidance_scale": 7.0,
            "scene_type": "morning_clouds",
        }, indent=2)
    },
    {
        "instruction": "First class luxury experience",
        "output": json.dumps({
            "prompt": "two people taking selfie in first class airplane suite, spacious luxury cabin, premium amenities visible, elegant lighting, sophisticated expressions, high-end travel experience, high quality photo, detailed, realistic",
            "negative_prompt": "cheap, low quality, economy, cramped, ugly",
            "ip_adapter_scale": 0.6,
            "guidance_scale": 7.5,
            "scene_type": "first_class",
        }, indent=2)
    },
    {
        "instruction": "Takeoff excitement from runway",
        "output": json.dumps({
            "prompt": "two friends taking selfie during airplane takeoff, runway visible through window, excited nervous expressions, beginning of journey mood, window seat, high quality photo, realistic, dynamic moment",
            "negative_prompt": "calm, boring, ugly, blurry, static",
            "ip_adapter_scale": 0.7,
            "guidance_scale": 7.5,
            "scene_type": "takeoff",
        }, indent=2)
    },
    {
        "instruction": "Red eye flight tired but happy",
        "output": json.dumps({
            "prompt": "two travelers taking selfie during red eye flight, tired but happy expressions, blankets visible, dim cabin lighting, night atmosphere, cozy travel mood, high quality photo, realistic",
            "negative_prompt": "energetic, bright, ugly, blurry",
            "ip_adapter_scale": 0.65,
            "guidance_scale": 7.0,
            "scene_type": "red_eye",
        }, indent=2)
    },
    {
        "instruction": "Landing with destination airport view",
        "output": json.dumps({
            "prompt": "two people taking selfie during airplane landing, destination airport visible through window, excited arrival expressions, end of journey celebration, window seat, high quality photo, realistic, arrival mood",
            "negative_prompt": "departure, ugly, blurry, bad quality",
            "ip_adapter_scale": 0.65,
            "guidance_scale": 7.5,
            "scene_type": "landing",
        }, indent=2)
    },
    {
        "instruction": "Ocean view tropical destination",
        "output": json.dumps({
            "prompt": "two friends taking selfie in airplane, tropical ocean and islands visible through window, bright blue water below, vacation excitement mood, happy expressions, window seat, high quality photo, realistic, travel adventure",
            "negative_prompt": "ugly, blurry, dark, winter, mountains",
            "ip_adapter_scale": 0.65,
            "guidance_scale": 7.0,
            "scene_type": "tropical",
        }, indent=2)
    },
    {
        "instruction": "Business trip colleagues professional",
        "output": json.dumps({
            "prompt": "two business colleagues taking selfie in airplane, professional friendly expressions, business casual attire, business travel atmosphere, modern cabin, high quality photo, realistic, professional mood",
            "negative_prompt": "casual, party, ugly, blurry, unprofessional",
            "ip_adapter_scale": 0.6,
            "guidance_scale": 7.5,
            "scene_type": "business",
        }, indent=2)
    },
]

print(f"✅ Created {len(scene_planning_data)} training examples")

In [ ]:
# Cell 6: Fine-tune TinyLlama for Scene Planning
!pip install -q unsloth

from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import json

print("🔧 Fine-tuning TinyLlama for scene planning...")

# Load TinyLlama with 4-bit quantization
print("  Loading TinyLlama...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/tinyllama-bnb-4bit",
    max_seq_length=2048,
    load_in_4bit=True,
)

# Add LoRA adapters
print("  Adding LoRA adapters...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
)

# Format training data
def format_prompt(item):
    return f"""<|system|>
You are an inflight selfie scene planner. Given a user's description, output a JSON configuration with optimal parameters for generating a realistic inflight selfie.
<|user|>
{item['instruction']}
<|assistant|>
{item['output']}"""

formatted_data = [{"text": format_prompt(item)} for item in scene_planning_data]
dataset = Dataset.from_list(formatted_data)

print(f"  Training on {len(dataset)} examples...")

# Train
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    args=TrainingArguments(
        output_dir="./scene_planner_lora",
        num_train_epochs=10,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=5,
        save_steps=50,
        warmup_steps=10,
    ),
)

trainer.train()

# Save the model
print("  Saving model...")
model.save_pretrained("scene_planner_lora")
tokenizer.save_pretrained("scene_planner_lora")

print("✅ Scene planner trained and saved!")

In [ ]:
# Cell 7: Create Complete Pipeline with Scene Planner
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os

class CompleteInflightSelfiePipeline:
    """
    Complete pipeline with:
    1. TinyLlama Scene Planner - generates optimal parameters
    2. InsightFace - extracts face embeddings
    3. SDXL + IP-Adapter-FaceID - generates image
    """
    
    def __init__(self):
        print("🚀 Initializing Complete Inflight Selfie Pipeline...")
        self.setup_scene_planner()
        self.setup_image_generator()
        print("✅ Complete pipeline ready!\n")
    
    def setup_scene_planner(self):
        """Load fine-tuned TinyLlama."""
        print("  Loading scene planner...")
        
        if os.path.exists("scene_planner_lora"):
            # Load base model
            base_model = AutoModelForCausalLM.from_pretrained(
                "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                torch_dtype=torch.float16,
                device_map="auto"
            )
            
            # Load LoRA weights
            self.scene_planner = PeftModel.from_pretrained(base_model, "scene_planner_lora")
            self.scene_tokenizer = AutoTokenizer.from_pretrained("scene_planner_lora")
            print("    ✓ Loaded fine-tuned scene planner")
        else:
            # Fallback to base model
            self.scene_planner = AutoModelForCausalLM.from_pretrained(
                "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                torch_dtype=torch.float16,
                device_map="auto"
            )
            self.scene_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
            print("    ⚠️ Using base TinyLlama (not fine-tuned)")
    
    def setup_image_generator(self):
        """Initialize the IP-Adapter-FaceID pipeline."""
        print("  Loading image generator...")
        self.image_pipeline = pipeline  # Use the already initialized pipeline
        print("    ✓ Image generator ready")
    
    def plan_scene(self, user_prompt: str) -> dict:
        """Generate scene parameters from user prompt using TinyLlama."""
        
        full_prompt = f"""<|system|>
You are an inflight selfie scene planner. Given a user's description, output a JSON configuration with optimal parameters for generating a realistic inflight selfie.
<|user|>
{user_prompt}
<|assistant|>
"""
        
        inputs = self.scene_tokenizer(full_prompt, return_tensors="pt").to("cuda")
        outputs = self.scene_planner.generate(
            **inputs,
            max_new_tokens=400,
            temperature=0.7,
            do_sample=True,
            pad_token_id=self.scene_tokenizer.eos_token_id
        )
        response = self.scene_tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract JSON from response
        try:
            json_start = response.rfind("{")
            json_end = response.rfind("}") + 1
            
            if json_start >= 0 and json_end > json_start:
                params = json.loads(response[json_start:json_end])
                return params
        except:
            pass
        
        # Fallback defaults
        return {
            "prompt": f"two people taking selfie in airplane, {user_prompt}, high quality photo, realistic, detailed faces",
            "negative_prompt": "ugly, blurry, distorted, bad anatomy, deformed",
            "ip_adapter_scale": 0.65,
            "guidance_scale": 7.5,
        }
    
    def generate(
        self,
        user_prompt: str,
        person1_images: List,
        person2_images: List,
        seed: int = None,
    ) -> Image.Image:
        """
        Complete generation pipeline.
        
        Args:
            user_prompt: Natural language scene description
            person1_images: List of images for person 1
            person2_images: List of images for person 2
            seed: Random seed for reproducibility
        
        Returns:
            Generated PIL Image
        """
        
        print("\n" + "="*60)
        print("🎬 GENERATING INFLIGHT SELFIE")
        print("="*60)
        
        # Step 1: Plan the scene
        print("\n📋 Step 1: Planning scene...")
        scene_params = self.plan_scene(user_prompt)
        print(f"  Scene type: {scene_params.get('scene_type', 'custom')}")
        print(f"  Prompt: {scene_params['prompt'][:80]}...")
        print(f"  IP Scale: {scene_params.get('ip_adapter_scale', 0.65)}")
        print(f"  CFG Scale: {scene_params.get('guidance_scale', 7.5)}")
        
        # Step 2: Generate image
        print("\n🎨 Step 2: Generating image...")
        result_image = self.image_pipeline.generate_selfie(
            person1_images=person1_images,
            person2_images=person2_images,
            prompt=scene_params["prompt"],
            negative_prompt=scene_params.get("negative_prompt"),
            ip_adapter_scale=scene_params.get("ip_adapter_scale", 0.65),
            guidance_scale=scene_params.get("guidance_scale", 7.5),
            num_inference_steps=30,
            seed=seed,
        )
        
        print("\n" + "="*60)
        print("✅ GENERATION COMPLETE!")
        print("="*60 + "\n")
        
        return result_image


# Initialize complete pipeline
print("\n" + "="*60)
complete_pipeline = CompleteInflightSelfiePipeline()
print("="*60 + "\n")

## Phase 4: Testing & Demo

In [ ]:
# Cell 8: Upload Test Images and Generate
from google.colab import files
from IPython.display import display
import os

print("📤 Upload face images for testing...\n")

print("👤 Person 1: Upload 1-5 face images")
uploaded_p1 = files.upload()
person1_images = list(uploaded_p1.keys())
print(f"  ✓ Uploaded {len(person1_images)} images\n")

print("👤 Person 2: Upload 1-5 face images")
uploaded_p2 = files.upload()
person2_images = list(uploaded_p2.keys())
print(f"  ✓ Uploaded {len(person2_images)} images\n")

# Test scene descriptions
test_scenes = [
    "Two friends taking a sunset selfie flying to Dubai",
    "Business class celebration with champagne",
    "Night flight with city lights below",
    "Morning flight over fluffy clouds",
]

print("\n🎬 Choose a scene or enter your own:")
for i, scene in enumerate(test_scenes, 1):
    print(f"  {i}. {scene}")

scene_choice = input("\nEnter number (1-4) or custom description: ")

if scene_choice.isdigit() and 1 <= int(scene_choice) <= 4:
    user_prompt = test_scenes[int(scene_choice) - 1]
else:
    user_prompt = scene_choice

print(f"\n✨ Generating: {user_prompt}")

# Generate!
result = complete_pipeline.generate(
    user_prompt=user_prompt,
    person1_images=person1_images,
    person2_images=person2_images,
    seed=42,  # For reproducibility
)

# Display result
print("\n📸 Generated Image:")
display(result)

# Save result
result.save("generated_inflight_selfie.png")
print("\n💾 Saved as: generated_inflight_selfie.png")

# Download
files.download("generated_inflight_selfie.png")

In [ ]:
# Cell 9: Interactive Gradio Demo
import gradio as gr

def generate_selfie_gradio(
    person1_img1, person1_img2, person1_img3,
    person2_img1, person2_img2, person2_img3,
    prompt,
    seed,
):
    """Gradio interface for generation."""
    
    # Collect uploaded images
    p1_images = [img for img in [person1_img1, person1_img2, person1_img3] if img is not None]
    p2_images = [img for img in [person2_img1, person2_img2, person2_img3] if img is not None]
    
    if not p1_images or not p2_images:
        return None, "⚠️ Please upload at least one image for each person!"
    
    try:
        result = complete_pipeline.generate(
            user_prompt=prompt,
            person1_images=p1_images,
            person2_images=p2_images,
            seed=seed if seed > 0 else None,
        )
        return result, "✅ Generation successful!"
    except Exception as e:
        return None, f"❌ Error: {str(e)}"

# Create Gradio interface
demo = gr.Interface(
    fn=generate_selfie_gradio,
    inputs=[
        gr.Image(type="pil", label="Person 1 - Image 1"),
        gr.Image(type="pil", label="Person 1 - Image 2 (optional)"),
        gr.Image(type="pil", label="Person 1 - Image 3 (optional)"),
        gr.Image(type="pil", label="Person 2 - Image 1"),
        gr.Image(type="pil", label="Person 2 - Image 2 (optional)"),
        gr.Image(type="pil", label="Person 2 - Image 3 (optional)"),
        gr.Textbox(
            label="Scene Description",
            placeholder="e.g., Two friends taking a sunset selfie flying to Dubai",
            value="Two friends taking a sunset selfie flying to Dubai"
        ),
        gr.Slider(minimum=-1, maximum=10000, value=42, step=1, label="Seed (-1 for random)"),
    ],
    outputs=[
        gr.Image(type="pil", label="Generated Inflight Selfie"),
        gr.Textbox(label="Status"),
    ],
    title="✈️ Inflight Selfie Generator",
    description="Upload photos of two people and describe your dream inflight selfie scene!",
    examples=[
        [None, None, None, None, None, None, "Two friends taking a sunset selfie flying to Dubai", 42],
        [None, None, None, None, None, None, "Business class celebration with champagne", 123],
        [None, None, None, None, None, None, "Night flight with city lights below", 456],
    ],
)

# Launch
demo.launch(share=True, debug=True)

## Phase 5: Export Models for Production

In [ ]:
# Cell 10: Export All Models
import shutil
from pathlib import Path

print("📦 Exporting models for production...\n")

# Create export directory
EXPORT_DIR = Path("./inflight_selfie_export")
EXPORT_DIR.mkdir(exist_ok=True)

# Export scene planner
if os.path.exists("scene_planner_lora"):
    print("  Copying scene planner LoRA weights...")
    shutil.copytree("scene_planner_lora", EXPORT_DIR / "scene_planner_lora", dirs_exist_ok=True)
    print("    ✓ Scene planner exported")

# Create README
readme_content = """# Inflight Selfie Generator - Exported Models

## Contents

- `scene_planner_lora/` - Fine-tuned TinyLlama LoRA weights for scene planning

## Base Models Required

These will be downloaded automatically by the pipeline:

1. **SDXL Base**: `stabilityai/stable-diffusion-xl-base-1.0`
2. **IP-Adapter-FaceID**: `h94/IP-Adapter-FaceID`
3. **TinyLlama Base**: `TinyLlama/TinyLlama-1.1B-Chat-v1.0`
4. **InsightFace**: antelopev2 models

## Usage

1. Extract this archive
2. Place in your project's `models/` directory
3. The pipeline will automatically load the LoRA weights

## Training Details

- **Scene Planner**: TinyLlama + LoRA (r=16)
- **Training Data**: 10 inflight scene examples
- **Training Time**: ~20 minutes on T4 GPU
- **Framework**: Unsloth + TRL
"""

with open(EXPORT_DIR / "README.md", "w") as f:
    f.write(readme_content)

print("\n  Creating archive...")
!zip -r inflight_selfie_models.zip {EXPORT_DIR}

print("\n✅ Export complete!\n")
print("📥 Downloading archive...")
files.download("inflight_selfie_models.zip")

print("\n" + "="*60)
print("✨ Models exported successfully!")
print("="*60)

## Summary

This notebook has:

1. ✅ Set up IP-Adapter-FaceID with SDXL
2. ✅ Fine-tuned TinyLlama for scene planning
3. ✅ Created complete generation pipeline
4. ✅ Provided testing and demo interfaces
5. ✅ Exported models for production

### Next Steps

1. Download the exported models zip
2. Set up the FastAPI backend (see `server.py`)
3. Build the Next.js frontend
4. Deploy to production

### Performance Tips

- Use multiple face images per person for better results
- Adjust `ip_adapter_scale` (0.5-0.8) to balance identity vs scene quality
- Higher `guidance_scale` (8-10) for more prompt adherence
- Use seeds for reproducible results

### Credits

- IP-Adapter-FaceID: Tencent AI Lab
- Stable Diffusion XL: Stability AI
- InsightFace: Jia Guo, Jiankang Deng
- TinyLlama: Zhang et al.
